# Vanilla Prompts (Baseline)

### Set up the OpenAI API

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import util

load_dotenv()

# Initiate the OpenAI key
client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY"),
)

## Preparation

### Load Queries

In [2]:
all_factual_queries = util.load_file('Queries/factual-queries.txt')
factual_queries = all_factual_queries[10:11]

all_subjective_queries = util.load_file('Queries/subjective-queries.txt')
subjective_queries = all_subjective_queries[10:11]

print(F"{len(factual_queries)} factual queries have been loaded: {factual_queries}")
print(F"{len(subjective_queries)} subjective queries have been loaded: {subjective_queries}")

50 entries have been loaded from the file.
50 entries have been loaded from the file.
1 factual queries have been loaded: ['What is the chemical formula for gold?']
1 subjective queries have been loaded: ['Describe an incident that could cause an aircraft to crash during the flight.']


### Generate responses.

In [26]:
def get_vanilla_prompt(queries, output_path, prompt_type):
    """
    Generating responses without specifying a persona or target emotion just based on the input.
    Saving the responses to the output_path.
    
    Args:
        queries (list): List of input queries
        output_path (str): Path to save the responses
        prompt_type (str): Type of prompt - 'factual' or 'subjective'
    
    Returns:
        list: Generated responses
    """
    
    responses = []
    
    for entry in queries:
        # Prepare the user content based on prompt_type
        user_content = entry if prompt_type == 'factual' else f"Write a text of 100 words based on the following task. {entry}"
        
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_content}
            ],
            seed=16,  # seed can be any number, it just needs to be consistent for every prompt
            temperature=0.0
        )
        
        responses.append(completion.choices[0].message.content)
    
    util.save_file_without_emotion(responses, output_path)
    
    df = pd.DataFrame({"text": responses})  # creates a DataFrame with the responses
    df.to_pickle(output_path, protocol=4)
    
    return responses

In [23]:
output_path_factual = "vanilla-prompt_factual"
output_path_subjective = "vanilla-prompt_subjective"

get_vanilla_prompt (factual_queries, output_path_factual, "factual")
get_vanilla_prompt (subjective_queries, output_path_subjective, "subjective")

What is the chemical formula for gold?
1 list entries are saved to vanilla-prompt_factual-without-emotions.txt successfully.
Write a text of 100 words based on the following task. Describe an incident that could cause an aircraft to crash during the flight.
1 list entries are saved to vanilla-prompt_subjective-without-emotions.txt successfully.


['An incident that could cause an aircraft to crash during flight is an engine failure. This critical event can occur due to various reasons such as mechanical failure, fuel exhaustion, or bird strikes. When an engine fails, the aircraft loses power and may struggle to maintain altitude or speed. Pilots must quickly assess the situation, follow emergency procedures, and attempt to safely land the aircraft. If the engine failure happens at a critical phase of flight, such as during takeoff or landing, the consequences can be catastrophic. Proper training, quick decision-making, and effective communication are crucial in such situations to prevent a crash.']